In [1]:
from dataset import SchoolingDataset
import logging

/Users/mkrtichhovsepyan/Desktop/AUTO/acim-happiness/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# logging configuration
logging.basicConfig(
    level=logging.INFO,
    format="%(name)s:%(lineno)s [%(levelname)s]: %(message)s",
)

In [3]:
SCHOOLING_DATA = SchoolingDataset.from_kaggle()

In [4]:
SCHOOLING_DATA[2012]

,ISO3,Country,Continent,Hemisphere,Human Development Groups,UNDP Developing Regions,HDI Rank (2021),Expected Years of Schooling (2012)
0,AFG,Afghanistan,Asia,Northern Hemisphere,Low,SA,180.0,9.577323
1,AGO,Angola,Africa,Southern Hemisphere,Medium,SSA,148.0,10.082500
2,ALB,Albania,Europe,Northern Hemisphere,High,ECA,67.0,14.586650
3,AND,Andorra,Europe,Northern Hemisphere,Very High,NaN,40.0,13.524016
4,ARE,United Arab Emirates,Asia,Northern Hemisphere,Very High,AS,26.0,12.945893
...,...,...,...,...,...,...,...,...
190,WSM,Samoa,Oceania,Southern Hemisphere,High,EAP,111.0,12.252976
191,YEM,Yemen,Asia,Northern Hemisphere,Low,AS,183.0,9.098710
192,ZAF,South Africa,Africa,Southern Hemisphere,High,SSA,109.0,13.497650
193,ZMB,Zambia,Africa,Southern Hemisphere,Medium,SSA,154.0,10.925998
